In [8]:
%matplotlib
from ipywidgets import interact
import paho.mqtt.client as mqtt
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
import time as time
from matplotlib.widgets import Button, CheckButtons
import json
message_buffer=[]
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("SmartDemoTable2/Jupyter/Outgoing")

def on_message(client, userdata, msg):
    json_data=msg.payload.decode('utf-8')
    print(json_data)
    message_buffer.append(json_data)

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com")
client.loop_start()
client.publish("SmartDemoTable2/Jupyter/Ingoing","summation")
while not message_buffer:
    pass
message=message_buffer.pop(0)
df=pd.read_json(message)
print(df)
message_buffer.clear()

Connected with result code 0
{"Generation":{"now":21.392},"Load":{"now":22.092},"Storage":{"now":0.7}}
                            Generation    Load  Storage
2023-11-21 11:40:56.243301      21.392  22.092      0.7


C:\Users\Thijs Laptop\AppData\Local\Temp\ipykernel_22036\1681239735.py:30: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df=pd.read_json(message)


In [11]:
sns.set_theme()
client.publish("SmartDemoTable2/Jupyter/Ingoing","module generation Table1")
while not message_buffer:
    pass
df=pd.read_json(message_buffer.pop(0))
df.plot

{"Table1_1071771887_Gen0":{"now":15.192}}


C:\Users\Thijs Laptop\AppData\Local\Temp\ipykernel_22036\3193810919.py:5: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df=pd.read_json(message_buffer.pop(0))


In [11]:
sns.set_theme(style="darkgrid")
dropdown=widgets.Dropdown(
    options=['HV','MV','LV'],
    description='Select voltage:',
    value='HV',
    disabled=False,
)
figure,ax=plt.subplots()
def onpick(event):
    artist = event.artist
    xmouse, ymouse = event.mouseevent.xdata, event.mouseevent.ydata
    x, y = artist.get_xdata(), artist.get_ydata()
    ind = event.ind
    message="index "+str(ind[0])
    print(message)
def f(x):
    global df
    message_buffer.clear()
    buffer="voltage "+x
    client.publish("SmartDemoTable2/Jupyter/Ingoing",buffer)
    while not message_buffer:
        pass
    message=message_buffer.pop(0)
    df=pd.read_json(message)
    ax.clear()
    df.plot(kind='line',ax=ax,picker=4)
    
ax.figure.canvas.mpl_connect("pick_event", onpick)
interact(f,x=dropdown)

interactive(children=(Dropdown(description='Select voltage:', options=('HV', 'MV', 'LV'), value='HV'), Output(…

<function __main__.f(x)>

index 20
